In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, WeightedRandomSampler, random_split, Dataset
from torchvision import transforms, datasets, models
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from collections import Counter
from PIL import Image
import os
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt

# Step 1: Define the ResNet18 model with Grad-CAM support
class ResNetWithGradCAM(nn.Module):
    def __init__(self, num_classes):
        super(ResNetWithGradCAM, self).__init__()
        self.model = models.resnet18(pretrained=True)
        self.num_features = self.model.fc.in_features
        self.model.fc = nn.Linear(self.num_features, num_classes)  # Modify the final layer

        # Register hooks to capture activations and gradients
        self.gradients = None
        self.activations = None
        self.model.layer4.register_forward_hook(self.activations_hook)
        self.model.layer4.register_backward_hook(self.gradients_hook)

    def activations_hook(self, module, input, output):
        self.activations = output

    def gradients_hook(self, module, grad_input, grad_output):
        self.gradients = grad_output[0]

    def forward(self, x):
        return self.model(x)

    def get_activations(self):
        return self.activations

    def get_gradients(self):
        return self.gradients

# Step 2: Define the device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Step 3: Define transformations
standard_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

augmentation_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Step 4: Custom dataset class
class CustomDataset(datasets.ImageFolder):
    def __init__(self, root, standard_transform=None, augmentation_transform=None):
        super().__init__(root)
        self.standard_transform = standard_transform
        self.augmentation_transform = augmentation_transform

    def __getitem__(self, index):
        path, label = self.imgs[index]
        image = Image.open(path).convert("RGB")

        if label == 1 and self.augmentation_transform:
            image = self.augmentation_transform(image)
        else:
            image = self.standard_transform(image)

        return image, label

# Step 5: Define the UnlabeledDataset class
class UnlabeledDataset(Dataset):
    def __init__(self, root, transform=None):
        self.root = root
        self.transform = transform
        self.image_paths = sorted([os.path.join(root, fname) for fname in os.listdir(root) if fname.endswith(('.jpg', '.png', '.jpeg'))])

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, os.path.basename(image_path)

# Step 6: Load the training dataset
train_data_dir = "/content/drive/MyDrive/E-RAU(DB)/MA680/data/Shooting/Processed_Frames/TrainingImages"
train_dataset = CustomDataset(
    root=train_data_dir,
    standard_transform=standard_transform,
    augmentation_transform=augmentation_transform
)

print(f"Total dataset size: {len(train_dataset)}")

# Step 7: Calculate class weights
class_counts = Counter([label for _, label in train_dataset])
total_samples = sum(class_counts.values())
class_weights = [total_samples / class_counts[i] for i in range(len(class_counts))]
class_weights = torch.tensor(class_weights, dtype=torch.float32).to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights)

print(f"Class weights: {class_weights}")

# Step 8: Calculate sample weights
sample_weights = [class_weights[label] for _, label in train_dataset]
print(f"Sample weights length: {len(sample_weights)}")

# Step 9: Split the dataset
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
print(f"Train size: {train_size}, Validation size: {val_size}")

train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

# Step 10: Create sampler for training subset
train_indices = train_dataset.indices
train_sample_weights = [sample_weights[i] for i in train_indices]
sampler = WeightedRandomSampler(train_sample_weights, num_samples=len(train_dataset), replacement=True)

# Step 11: Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, sampler=sampler)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Step 12: Initialize model
model = ResNetWithGradCAM(num_classes=2).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

# Step 13: Training loop
num_epochs = 10
best_val_loss = float('inf')
patience = 3
trigger_times = 0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    scheduler.step()
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}")

    # Validation
    model.eval()
    val_loss = 0.0
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    val_loss /= len(val_loader)
    val_accuracy = accuracy_score(all_labels, all_preds)
    val_precision = precision_score(all_labels, all_preds, average='binary', pos_label=1)
    val_recall = recall_score(all_labels, all_preds, average='binary', pos_label=1)
    val_f1 = f1_score(all_labels, all_preds, average='binary', pos_label=1)
    print(f"Validation Loss: {val_loss:.4f}, "
          f"Accuracy: {val_accuracy:.4f}, Precision: {val_precision:.4f}, "
          f"Recall: {val_recall:.4f}, F1: {val_f1:.4f}")

    # Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        trigger_times = 0
        torch.save(model.state_dict(), "best_pretrained_model.pth")
    else:
        trigger_times += 1
        if trigger_times >= patience:
            print("Early stopping!")
            break

print("Training complete!")


In [ ]:


# Step 14: Enhanced Dual-Neuron Grad-CAM Implementation
def grad_cam_dual_neuron(model, image):
    """Generate Grad-CAM heatmaps for both classes with proper gradient flow"""
    model.eval()
    image = image.unsqueeze(0).to(device)
    image.requires_grad = True

    # Forward pass
    output = model(image)
    probs = torch.softmax(output, dim=1)

    # Zero gradients
    model.zero_grad()

    # Create one-hot encodings for both classes
    one_hot_pos = torch.zeros_like(output)
    one_hot_pos[0, 1] = 1  # Positive class (good form)

    one_hot_neg = torch.zeros_like(output)
    one_hot_neg[0, 0] = 1  # Negative class (bad form)

    # Backpropagate for positive class
    output.backward(gradient=one_hot_pos, retain_graph=True)
    gradients_pos = model.get_gradients().cpu().detach().numpy()
    activations = model.get_activations().cpu().detach().numpy()

    # Compute weights for positive class
    weights_pos = np.mean(gradients_pos, axis=(2, 3))

    # Create heatmap for positive class
    heatmap_pos = np.zeros(activations.shape[2:], dtype=np.float32)
    for i in range(activations.shape[1]):
        heatmap_pos += weights_pos[0, i] * activations[0, i]
    heatmap_pos = np.maximum(heatmap_pos, 0)
    heatmap_pos = (heatmap_pos - heatmap_pos.min()) / (heatmap_pos.max() - heatmap_pos.min() + 1e-8)

    # Backpropagate for negative class
    model.zero_grad()
    output.backward(gradient=one_hot_neg)
    gradients_neg = model.get_gradients().cpu().detach().numpy()

    # Compute weights for negative class
    weights_neg = np.mean(gradients_neg, axis=(2, 3))

    # Create heatmap for negative class
    heatmap_neg = np.zeros(activations.shape[2:], dtype=np.float32)
    for i in range(activations.shape[1]):
        heatmap_neg += weights_neg[0, i] * activations[0, i]
    heatmap_neg = np.maximum(heatmap_neg, 0)
    heatmap_neg = (heatmap_neg - heatmap_neg.min()) / (heatmap_neg.max() - heatmap_neg.min() + 1e-8)

    return heatmap_pos, heatmap_neg, probs[0].cpu().detach().numpy()

def visualize_dual_gradcam(image, heatmap_pos, heatmap_neg, pred_prob):
    """Visualize both class heatmaps side by side with original image"""
    # Convert tensor to numpy if needed
    if isinstance(image, torch.Tensor):
        image = image.cpu().numpy()
        image = np.transpose(image, (1, 2, 0))
        # Denormalize
        mean = np.array([0.485, 0.456, 0.406])
        std = np.array([0.229, 0.224, 0.225])
        image = std * image + mean
        image = np.clip(image, 0, 1)
        image = (image * 255).astype(np.uint8)

    # Create figure
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 6))

    # Show original image
    ax1.imshow(image)
    ax1.set_title(f"Original Image\n(True Class: {'Good' if pred_prob[1] > 0.5 else 'Bad'})")
    ax1.axis('off')

    # Prepare heatmaps
    heatmap_pos = cv2.resize(heatmap_pos, (image.shape[1], image.shape[0]))
    heatmap_neg = cv2.resize(heatmap_neg, (image.shape[1], image.shape[0]))

    # Create colored heatmaps
    heatmap_pos_viz = cv2.applyColorMap(np.uint8(255 * heatmap_pos), cv2.COLORMAP_JET)
    heatmap_neg_viz = cv2.applyColorMap(np.uint8(255 * heatmap_neg), cv2.COLORMAP_JET)

    # Create overlays
    overlay_pos = cv2.addWeighted(image, 0.5, heatmap_pos_viz, 0.5, 0)
    overlay_neg = cv2.addWeighted(image, 0.5, heatmap_neg_viz, 0.5, 0)

    # Show positive class heatmap
    ax2.imshow(overlay_pos)
    ax2.set_title(f"Good Form Heatmap\n(Prob: {pred_prob[1]:.2f})")
    ax2.axis('off')

    # Show negative class heatmap
    ax3.imshow(overlay_neg)
    ax3.set_title(f"Bad Form Heatmap\n(Prob: {pred_prob[0]:.2f})")
    ax3.axis('off')

    plt.tight_layout()
    plt.show()

# Step 15: Test the Grad-CAM visualization
sample_image, sample_label = train_dataset[0]
heatmap_pos, heatmap_neg, probs = grad_cam_dual_neuron(model, sample_image)
visualize_dual_gradcam(sample_image, heatmap_pos, heatmap_neg, probs)

# Step 16: Test dataset evaluation
test_data_dir = "/content/drive/MyDrive/E-RAU(DB)/MA680/data/Shooting/Processed_Frames/X.Test/Frames"
test_dataset = UnlabeledDataset(root=test_data_dir, transform=standard_transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Load best model
model.load_state_dict(torch.load("best_pretrained_model.pth"))
model.eval()

# Initialize lists
all_preds = []
all_filenames = []

# Run predictions
with torch.no_grad():
    for images, filenames in test_loader:
        images = images.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_filenames.extend(filenames)

# Save predictions
results = pd.DataFrame({"Filename": all_filenames, "Prediction": all_preds})
results.to_csv("test_predictions.csv", index=False)
print("Test predictions saved to test_predictions.csv")

# Step 17: Visualize Grad-CAM for test images
num_samples = 100  # Number of test images to visualize
for i in range(min(num_samples, len(test_dataset))):
    sample_image, sample_filename = test_dataset[i]
    heatmap_pos, heatmap_neg, probs = grad_cam_dual_neuron(model, sample_image)
    visualize_dual_gradcam(sample_image, heatmap_pos, heatmap_neg, probs)
    print(f"Filename: {sample_filename}, Prediction: {'Good' if all_preds[i] == 1 else 'Bad'}")